In [29]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
"""
check all regions au = nem + wem (swis)
"""
from notebook_init import settings
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import date
from datetime import datetime, timedelta
from opennem.db import get_database_engine, db_connect 
from opennem.schema.network import NetworkNEM
from opennem.api.stats.loader import load_statset
from opennem.api.stats.schema import OpennemData, OpennemDataSet
from opennem.utils.http import http
from opennem.utils.series import series_joined, series_are_equal


In [84]:

URL_ALL_REGIONS = "https://data.dev.opennem.org.au/v3/stats/au/AU/energy/2021.json"
URL_NEM = "https://data.dev.opennem.org.au/v3/stats/au/NEM/energy/2021.json"
URL_SWIS = "https://data.dev.opennem.org.au/v3/stats/au/WEM/energy/2021.json"

CHECK_DATE = (datetime.now() - timedelta(days=2)).date()

def get_df_for_feed(feed_url, code) -> None:
    r = http.get(feed_url)
    v3 = load_statset(r.json())
    _v = [(i.fuel_tech, get_date(i.history, CHECK_DATE)) for i in v3.data if i.id.endswith(".energy")]
    df = pd.DataFrame(_v, columns=["fueltech", f"value_{code}"]).set_index(["fueltech"])
    return df

def get_date(stat_set, dt: date):
    """Get for a date"""
    _values = stat_set.values()
    _get_value = list(filter(lambda x: x[0].date() == dt, _values))
    
    if not _get_value:
        return None

    return _get_value.pop()[1]

df = get_df_for_feed(URL_ALL_REGIONS, "au")

df_nem = get_df_for_feed(URL_NEM, "nem")
df_wem = get_df_for_feed(URL_SWIS, "wem")

df_compare = df.join(df_nem).join(df_wem)



In [87]:
df_compare["total"] = df_compare["value_nem"] + df_compare["value_wem"]
df_compare["eq"] = df_compare["total"] == df_compare["value_au"]
df_compare

,value_au,value_nem,value_wem,eq,total
fueltech,,,,,
bioenergy_biomass,0.5877,0.5877,NaN,False,NaN
gas_ccgt,17.1300,12.5600,4.5700,False,17.1300
coal_black,232.3000,219.3000,12.9300,False,232.2300
solar_rooftop,59.3500,50.2900,9.0640,False,59.3540
gas_wcmg,1.3550,1.3550,NaN,False,NaN
battery_discharging,0.2899,0.2899,NaN,False,NaN
coal_brown,76.7000,76.7000,NaN,False,NaN
gas_ocgt,12.2400,0.8733,11.3600,False,12.2333
pumps,5.8010,5.8010,NaN,False,NaN
